# Lab 4: Direct Lake Fallback

This lab assumes Lab 2 has been completed successfully and there is a Semantic Model named BigData_Model

## 1. Install Semantic Link Labs Python Library

In [ ]:
%pip install -q --disable-pip-version-check semantic-link-labs

## 2. Setup Parameters

In [ ]:
import sempy_labs as labs
from sempy import fabric
import sempy

LakehouseName = "BigData"
lakehouses = labs.list_lakehouses()["Lakehouse Name"]
for l in lakehouses:
    if l.startswith("Big"):
        LakehouseName = l

SemanticModelName = f"{LakehouseName}_model"

lakehouses=labs.list_lakehouses()["Lakehouse Name"]
if LakehouseName in lakehouses.values:
    lakehouseId = notebookutils.lakehouse.getWithProperties(LakehouseName)["id"]
else:
    print("You need to complete Lab 2 to create the required lakehouse for this lab")

workspaceId = notebookutils.lakehouse.getWithProperties(LakehouseName)["workspaceId"]
workspaceName = sempy.fabric.resolve_workspace_name(workspaceId)
print(f"WorkspaceId = {workspaceId}, LakehouseID = {lakehouseId}, Workspace Name = {workspaceName}")



## 3. Create function to run DAX queries with a server timings trace

In [ ]:
import warnings
import time
from Microsoft.AnalysisServices.Tabular import TraceEventArgs
from typing import Dict, List, Optional, Callable
import pandas

def runDMV():
    df = sempy.fabric.evaluate_dax(
        dataset=SemanticModelName, 
        dax_string="""
        
        SELECT 
            MEASURE_GROUP_NAME AS [TABLE],
            ATTRIBUTE_NAME AS [COLUMN],
            DATATYPE ,
            DICTIONARY_SIZE 		    AS SIZE ,
            DICTIONARY_ISPAGEABLE 		AS PAGEABLE ,
            DICTIONARY_ISRESIDENT		AS RESIDENT ,
            DICTIONARY_TEMPERATURE		AS TEMPERATURE,
            DICTIONARY_LAST_ACCESSED	AS LASTACCESSED 
        FROM $SYSTEM.DISCOVER_STORAGE_TABLE_COLUMNS 
        ORDER BY 
            [DICTIONARY_TEMPERATURE] DESC
        
        """)
    display(df)

def filter_func(e):
    retVal:bool=True
    if e.EventSubclass.ToString() == "VertiPaqScanInternal":
        retVal=False      
    #     #if e.EventSubClass.ToString() == "VertiPaqScanInternal":
    #     retVal=False
    return retVal

# define events to trace and their corresponding columns
def runQueryWithTrace (expr:str,workspaceName:str,SemanticModelName:str,Result:Optional[bool]=True,Trace:Optional[bool]=True,DMV:Optional[bool]=True,ClearCache:Optional[bool]=True) -> pandas.DataFrame :
    event_schema = fabric.Trace.get_default_query_trace_schema()
    event_schema.update({"ExecutionMetrics":["EventClass","TextData"]})
    del event_schema['VertiPaqSEQueryBegin']
    del event_schema['VertiPaqSEQueryCacheMatch']
    del event_schema['DirectQueryBegin']

    warnings.filterwarnings("ignore")

    WorkspaceName = workspaceName
    SemanticModelName = SemanticModelName

    if ClearCache:
        labs.clear_cache(SemanticModelName)

    with fabric.create_trace_connection(SemanticModelName,WorkspaceName) as trace_connection:
        # create trace on server with specified events
        with trace_connection.create_trace(
            event_schema=event_schema, 
            name="Simple Query Trace",
            filter_predicate=filter_func,
            stop_event="QueryEnd"
            ) as trace:

            trace.start()

            df=sempy.fabric.evaluate_dax(
                dataset=SemanticModelName, 
                dax_string=expr)

            if Result:
                displayHTML(f"<H2>####### DAX QUERY RESULT #######</H2>")
                display(df)

            # Wait 5 seconds for trace data to arrive
            time.sleep(5)

            # stop Trace and collect logs
            final_trace_logs = trace.stop()

    if Trace:
        displayHTML(f"<H2>####### SERVER TIMINGS #######</H2>")
        display(final_trace_logs)
    
    if DMV:
        displayHTML(f"<H2>####### SHOW DMV RESULTS #######</H2>")
        runDMV()

    return final_trace_logs


In [ ]:
runDMV()

## 4. Run query that involves a Direct Query Fallback

In [ ]:
trace1 = runQueryWithTrace(
    """
    EVALUATE
        SUMMARIZECOLUMNS(
                dim_Date[FirstDateofMonth] ,
                "Count of Transactions" , COUNTROWS(fact_myevents_1bln) ,
                "Sum of Sales (1bln)" , [Sum of Sales (1bln)] ,
                "Sum of Sales (2bln)" , [Sum of Sales (2bln)]
        )
        ORDER BY [FirstDateofMonth]
    """ , workspaceName , SemanticModelName
)
display(trace1)

## 5. Change Custom Semantic Model to be **Direct Lake Only**

In [ ]:
tom = labs.tom.TOMWrapper(dataset=SemanticModelName, workspace=workspaceName, readonly=False)
tom.set_direct_lake_behavior("DirectLakeOnly") ##  Can be set to any of ['Automatic', 'DirectLakeOnly', 'DirectQueryOnly'].
tom.model.SaveChanges()
print("Model changed")

## 6. Run query that involves a Direct Query Fallback

In [ ]:
from sempy import fabric
x = sempy.fabric._client._adomd_connection.FabricAdomdException
try:
    runQueryWithTrace(
        """
        EVALUATE
            SUMMARIZECOLUMNS(
                    dim_Date[FirstDateofMonth] ,
                    "Count of Transactions" , COUNTROWS(fact_myevents_1bln) ,
                    "Sum of Sales (1bln)" , [Sum of Sales (1bln)] ,
                    "Sum of Sales (2bln)" , [Sum of Sales (2bln)]
            )
            ORDER BY [FirstDateofMonth]
        """ , workspaceName , SemanticModelName
    )
except sempy.fabric._client._adomd_connection.FabricAdomdException as f:
    print(f)
except Exception as e:
    print(e)

## 7. Change Custom Semantic Model to be **Automatic**

In [ ]:
tom = labs.tom.TOMWrapper(dataset=SemanticModelName, workspace=workspaceName, readonly=False)
tom.set_direct_lake_behavior("Automatic") ##  ['Automatic', 'DirectLakeOnly', 'DirectQueryOnly'].
tom.model.SaveChanges()
print("Model changed")

## 8. Run query that involves a Direct Query Fallback

In [ ]:
trace2 = runQueryWithTrace(
    """
    EVALUATE
        SUMMARIZECOLUMNS(
                dim_Date[FirstDateofMonth] ,
                "Count of Transactions" , COUNTROWS(fact_myevents_1bln) ,
                "Sum of Sales (1bln)" , [Sum of Sales (1bln)] ,
                "Sum of Sales (2bln)" , [Sum of Sales (2bln)]
        )
        ORDER BY [FirstDateofMonth]
    """ , workspaceName , SemanticModelName, Trace=False, DMV=False
)
display(trace2)

## 9. Stop the Spark session

In [ ]:
mssparkutils.session.stop()